In [1]:
import numpy as np
import pandas as pd
import random
import datetime
import math
import pymongo
import string
import csv

In [2]:
df = pd.read_csv('simple_dataset.csv')

,artist,track,playlist,artist_track
0,Elvis Costello,The Angels Wanna Wear My Red Shoes,HARD ROCK 2010,Elvis Costello The Angels Wanna Wear My Red Shoes
1,Elvis Costello The Attractions,Whats So Funny Bout Peace Love And Understanding,HARD ROCK 2010,Elvis Costello The Attractions Whats So Funny...
2,Tiffany Page,7 Years Too Late,HARD ROCK 2010,Tiffany Page 7 Years Too Late
3,Elvis Costello The Attractions,Accidents Will Happen,HARD ROCK 2010,Elvis Costello The Attractions Accidents Will...
4,Elvis Costello,Alison,HARD ROCK 2010,Elvis Costello Alison


In [3]:
# Connection to mongoDB, SYP database 

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient['SYP2']

In [4]:
df_playlist = pd.DataFrame(df)
df_playlist = df_playlist.drop_duplicates(subset=['playlist']) # Take unique playlist name 
# Generate a random date_of_creation for each playlist

start_date = datetime.datetime(2010, 1, 1)
end_date = datetime.datetime(2020, 2, 1)
time_between_dates = end_date - start_date
days_between_dates = time_between_dates.days

df_playlist['date_of_creation'] = [ (start_date + datetime.timedelta(random.randrange(days_between_dates))) for k in df_playlist.playlist]
# Drop useless columns for playlists 
df_playlist_inserted = df_playlist.drop(['artist','track'],axis=1)
df_playlist_inserted.head()

,playlist,artist_track,date_of_creation
0,HARD ROCK 2010,Elvis Costello The Angels Wanna Wear My Red Shoes,2016-01-27
67,IOW 2012,Noah And The Whale 5 Years Time,2016-12-23
104,2080,2080 1 Up,2018-08-22
114,C418,C418 Alive,2013-09-07
148,Chill out,C418 0x10c 01,2015-09-14


In [5]:
playlist_coll = mydb['playlists'] # Select the playlists collection
playlist_dict = df_playlist_inserted.to_dict("records") # Transform the dataframe in a dict 
playlists_id = playlist_coll.insert_many(playlist_dict)

In [6]:
df_songs = pd.DataFrame(df)
df_songs['artist_track'] = pd.DataFrame((df['artist'] + " " + df['track']))
df_songs = df_songs.drop_duplicates(subset=['artist_track']) # Take unique songs combination artist_trackname 
df_songs_inserted = df_songs.drop(['artist_track','playlist'],axis=1) # Drop useless columns
df_songs_inserted.head()

,artist,track
0,Elvis Costello,The Angels Wanna Wear My Red Shoes
1,Elvis Costello The Attractions,Whats So Funny Bout Peace Love And Understanding
2,Tiffany Page,7 Years Too Late
3,Elvis Costello The Attractions,Accidents Will Happen
4,Elvis Costello,Alison


In [7]:
songs_coll = mydb['songs'] # Select the songs collection
song_dict = [{k:v for k,v in m.items() if pd.notnull(v)} for m in df_songs_inserted.to_dict(orient='records')]
songs_id = songs_coll.insert_many(song_dict) # returns the MongoDB ids for songs entities

In [8]:
df_unique_songs = list(df_songs['artist_track']) # Get unique artist_track values
df_unique_songs_id = songs_id.inserted_ids # Get unique songs_id values
zip_iterator = zip(df_unique_songs, df_unique_songs_id) 
key_value_dict = dict(zip_iterator) # Create a dict {artist_track: song_id}
df["song_id"] = df["artist_track"].map(key_value_dict) # Match artist_track with song_id
df.head()

,artist,track,playlist,artist_track,song_id
0,Elvis Costello,The Angels Wanna Wear My Red Shoes,HARD ROCK 2010,Elvis Costello The Angels Wanna Wear My Red Shoes,61eaa7a2f02e1050431fb76e
1,Elvis Costello The Attractions,Whats So Funny Bout Peace Love And Understanding,HARD ROCK 2010,Elvis Costello The Attractions Whats So Funny...,61eaa7a2f02e1050431fb76f
2,Tiffany Page,7 Years Too Late,HARD ROCK 2010,Tiffany Page 7 Years Too Late,61eaa7a2f02e1050431fb770
3,Elvis Costello The Attractions,Accidents Will Happen,HARD ROCK 2010,Elvis Costello The Attractions Accidents Will...,61eaa7a2f02e1050431fb771
4,Elvis Costello,Alison,HARD ROCK 2010,Elvis Costello Alison,61eaa7a2f02e1050431fb772


In [9]:
df_unique_playlist = list(df_playlist['playlist']) # Get unique playlist values
df_unique_playlist_id = playlists_id.inserted_ids # Get unique playlist_id values
zip_iterator = zip(df_unique_playlist, df_unique_playlist_id) 
key_value_dict = dict(zip_iterator) # Create a dict {playlist: playlist_id}
df["playlist_id"] = df["playlist"].map(key_value_dict) # Match playlist with playlist_id
df.head()

,artist,track,playlist,artist_track,song_id,playlist_id
0,Elvis Costello,The Angels Wanna Wear My Red Shoes,HARD ROCK 2010,Elvis Costello The Angels Wanna Wear My Red Shoes,61eaa7a2f02e1050431fb76e,61eaa79ff02e1050431f95af
1,Elvis Costello The Attractions,Whats So Funny Bout Peace Love And Understanding,HARD ROCK 2010,Elvis Costello The Attractions Whats So Funny...,61eaa7a2f02e1050431fb76f,61eaa79ff02e1050431f95af
2,Tiffany Page,7 Years Too Late,HARD ROCK 2010,Tiffany Page 7 Years Too Late,61eaa7a2f02e1050431fb770,61eaa79ff02e1050431f95af
3,Elvis Costello The Attractions,Accidents Will Happen,HARD ROCK 2010,Elvis Costello The Attractions Accidents Will...,61eaa7a2f02e1050431fb771,61eaa79ff02e1050431f95af
4,Elvis Costello,Alison,HARD ROCK 2010,Elvis Costello Alison,61eaa7a2f02e1050431fb772,61eaa79ff02e1050431f95af


In [10]:
# Get the list of songs contained in each playlist
group = df.drop_duplicates(['playlist_id','song_id']).reindex(columns=['song_id','artist','track','album','date','playlist_id']).rename(columns={"song_id": "_id"}).groupby('playlist_id')
series = group.apply(lambda x: [{k:v for k,v in m.items() if pd.notnull(v)} for m in x.to_dict(orient='records')])
df_playlist_song = pd.DataFrame({
    'playlist_id': series.index,
    'songs_in_playlist': series.values
})
df_playlist_song['songs_in_playlist'][1]

[{'_id': ObjectId('61eaa7a2f02e1050431fb7b1'),
  'artist': 'Noah And The Whale',
  'track': '5 Years Time',
  'playlist_id': ObjectId('61eaa79ff02e1050431f95b0')},
 {'_id': ObjectId('61eaa7a2f02e1050431fb7b2'),
  'artist': 'Noel Gallaghers High Flying Birds',
  'track': 'AKA What A Life',
  'playlist_id': ObjectId('61eaa79ff02e1050431f95b0')},
 {'_id': ObjectId('61eaa7a2f02e1050431fb7b3'),
  'artist': 'Pearl Jam',
  'track': 'Alive',
  'playlist_id': ObjectId('61eaa79ff02e1050431f95b0')},
 {'_id': ObjectId('61eaa7a2f02e1050431fb7b4'),
  'artist': 'Tom Petty And The Heartbreakers',
  'track': 'American Girl',
  'playlist_id': ObjectId('61eaa79ff02e1050431f95b0')},
 {'_id': ObjectId('61eaa7a2f02e1050431fb7b5'),
  'artist': 'Bruce Springsteen',
  'track': 'Atlantic City',
  'playlist_id': ObjectId('61eaa79ff02e1050431f95b0')},
 {'_id': ObjectId('61eaa7a2f02e1050431fb7b6'),
  'artist': 'Madness',
  'track': 'Baggy Trousers',
  'playlist_id': ObjectId('61eaa79ff02e1050431f95b0')},
 {'_id': 

In [11]:
def remove_song_values(x):
    for song in x:
        del song['playlist_id']
    return x 

In [12]:
df_playlist_song['songs_in_playlist'] = df_playlist_song['songs_in_playlist'].apply(remove_song_values) #Remove

In [13]:
df_playlist_song['songs_in_playlist'][0]

[{'_id': ObjectId('61eaa7a2f02e1050431fb76e'),
  'artist': 'Elvis Costello',
  'track': 'The Angels Wanna Wear My Red Shoes'},
 {'_id': ObjectId('61eaa7a2f02e1050431fb76f'),
  'artist': 'Elvis Costello  The Attractions',
  'track': 'Whats So Funny Bout Peace Love And Understanding'},
 {'_id': ObjectId('61eaa7a2f02e1050431fb770'),
  'artist': 'Tiffany Page',
  'track': '7 Years Too Late'},
 {'_id': ObjectId('61eaa7a2f02e1050431fb771'),
  'artist': 'Elvis Costello  The Attractions',
  'track': 'Accidents Will Happen'},
 {'_id': ObjectId('61eaa7a2f02e1050431fb772'),
  'artist': 'Elvis Costello',
  'track': 'Alison'},
 {'_id': ObjectId('61eaa7a2f02e1050431fb773'),
  'artist': 'Lissie',
  'track': 'All Be Okay'},
 {'_id': ObjectId('61eaa7a2f02e1050431fb774'),
  'artist': 'Paul McCartney',
  'track': 'Band On The Run'},
 {'_id': ObjectId('61eaa7a2f02e1050431fb775'),
  'artist': 'Joe Echo',
  'track': 'Beautiful'},
 {'_id': ObjectId('61eaa7a2f02e1050431fb776'),
  'artist': 'Paul McCartney',
 

In [14]:
# Add the column of embedded documents to the principal df and save it as "df_playlist_embedded"
df_playlist_embedded = (pd.merge(df.drop_duplicates(subset=['playlist_id']), df_playlist_song, on='playlist_id')) 

In [15]:
def insert_playlist_embedded(playlist_id,embedded_doc):
    playlist_coll.update_one(
        {"_id" : playlist_id}, 
        { "$set": { 'songs_in_playlist': embedded_doc}}
    )

In [16]:
%%capture
df_playlist_embedded.apply(lambda x: insert_playlist_embedded(x.playlist_id, x.songs_in_playlist,), axis=1)

In [17]:
# Get the list of playlists in which a song is contained, for each song
group = df.drop_duplicates(['playlist_id','song_id']).reindex(columns=['playlist_id','playlist','song_id']).rename(columns={"playlist_id": "_id", "playlist":"name"}).groupby('song_id')
series = group.apply(lambda x: x.to_dict('records'))
df_song_playlist = pd.DataFrame({
    'song_id': series.index,
    'playlists_that_contains_it': series.values
})
df_song_playlist.head()

,song_id,playlists_that_contains_it
0,61eaa7a2f02e1050431fb76e,"[{'_id': 61eaa79ff02e1050431f95af, 'name': 'HA..."
1,61eaa7a2f02e1050431fb76f,"[{'_id': 61eaa79ff02e1050431f95af, 'name': 'HA..."
2,61eaa7a2f02e1050431fb770,"[{'_id': 61eaa79ff02e1050431f95af, 'name': 'HA..."
3,61eaa7a2f02e1050431fb771,"[{'_id': 61eaa79ff02e1050431f95af, 'name': 'HA..."
4,61eaa7a2f02e1050431fb772,"[{'_id': 61eaa79ff02e1050431f95af, 'name': 'HA..."


In [18]:
#Remove values not needed 
def remove_playlist_values(x):
    for playlist in x:
        del playlist['song_id']
    return x 

In [19]:
#Remove values not needed
df_song_playlist['playlists_that_contains_it'] = df_song_playlist['playlists_that_contains_it'].apply(remove_playlist_values)  

In [20]:
df_song_playlist['playlists_that_contains_it'][0]

[{'_id': ObjectId('61eaa79ff02e1050431f95af'), 'name': 'HARD ROCK 2010'},
 {'_id': ObjectId('61eaa79ff02e1050431f95b9'), 'name': 'Starred'}]

In [21]:
# Add the column of embedded documents to the principal df and save it as "df_songs_embedded"
df_songs_embedded = (pd.merge(df.drop_duplicates(subset=['song_id']), df_song_playlist, on='song_id'))

In [22]:
def insert_songs_embedded(song_id,embedded_doc):
    songs_coll.update_one(
        {"_id" : song_id}, 
        { "$set": { 'playlists': embedded_doc } }
    )

In [23]:
%%capture
df_songs_embedded.apply(lambda x: insert_songs_embedded(x.song_id, x.playlists_that_contains_it), axis=1)

In [24]:
users_coll = mydb['users'] # Select the users collection

In [25]:
from random import randint
users = users_coll.find()
user = users[randint(0, 13000)]
print(user)

{'_id': ObjectId('61cc6cbb3bd8fb88c6629f30'), 'username': 'kike79', 'dateOfBirth': datetime.datetime(1955, 3, 16, 0, 0), 'dateOfCreation': datetime.datetime(2014, 6, 13, 0, 0), 'isAdmin': False, 'password': 'MgUoWPhDPNEkWEQI'}


In [26]:
def insert_creator_in_playlist(playlist_id, playlist_name):
    user = users[randint(0, 13000)]
    playlist_coll.update_one(
        {"_id" : playlist_id}, 
        { "$set": { 'creator': {'_id': user['_id'], 'username': user['username']}} }
    )
    users_coll.update_one(
        {"_id" : user['_id']}, 
        { "$push": { 'createdPlaylists': {'_id': playlist_id, 'name': playlist_name}} }
    )

In [27]:
df_playlist_embedded.apply(lambda x: insert_creator_in_playlist(x.playlist_id, x.playlist), axis=1)
df_playlist_embedded.head()

,artist,track,playlist,artist_track,song_id,playlist_id,songs_in_playlist
0,Elvis Costello,The Angels Wanna Wear My Red Shoes,HARD ROCK 2010,Elvis Costello The Angels Wanna Wear My Red Shoes,61eaa7a2f02e1050431fb76e,61eaa79ff02e1050431f95af,"[{'_id': 61eaa7a2f02e1050431fb76e, 'artist': '..."
1,Noah And The Whale,5 Years Time,IOW 2012,Noah And The Whale 5 Years Time,61eaa7a2f02e1050431fb7b1,61eaa79ff02e1050431f95b0,"[{'_id': 61eaa7a2f02e1050431fb7b1, 'artist': '..."
2,2080,1 Up,2080,2080 1 Up,61eaa7a2f02e1050431fb7d6,61eaa79ff02e1050431f95b1,"[{'_id': 61eaa7a2f02e1050431fb7d6, 'artist': '..."
3,C418,Alive,C418,C418 Alive,61eaa7a2f02e1050431fb7e0,61eaa79ff02e1050431f95b2,"[{'_id': 61eaa7a2f02e1050431fb7e0, 'artist': '..."
4,C418,0x10c 01,Chill out,C418 0x10c 01,61eaa7a2f02e1050431fb802,61eaa79ff02e1050431f95b3,"[{'_id': 61eaa7a2f02e1050431fb802, 'artist': '..."


In [28]:
playlist_coll.update_many( {}, { "$rename": { 
    "playlist": "name",
    "date_of_creation": "creationDate",
    "songs_in_playlist": "songs"
} } )

In [29]:
playlist_csv = df.drop_duplicates(subset=['playlist_id']).drop(['artist','track','artist_track'],axis=1).reindex(columns=['playlist_id','playlist'])
playlist_csv['playlist'] = playlist_csv['playlist'].apply(lambda x: str(x).replace('"',"'"))
playlist_csv.head()

,playlist_id,playlist
0,61eaa79ff02e1050431f95af,HARD ROCK 2010
67,61eaa79ff02e1050431f95b0,IOW 2012
104,61eaa79ff02e1050431f95b1,2080
114,61eaa79ff02e1050431f95b2,C418
148,61eaa79ff02e1050431f95b3,Chill out


In [30]:
songs_csv = df.drop_duplicates(subset=['song_id']).drop(['playlist','artist_track','playlist_id'],axis=1).reindex(columns=['track','song_id','artist'])
songs_csv['track'] = songs_csv['track'].apply(lambda x: str(x).replace('"',"'"))
songs_csv['artist'] = songs_csv['artist'].apply(lambda x: str(x).replace('"',"'"))
songs_csv.head()

,track,song_id,artist
0,The Angels Wanna Wear My Red Shoes,61eaa7a2f02e1050431fb76e,Elvis Costello
1,Whats So Funny Bout Peace Love And Understanding,61eaa7a2f02e1050431fb76f,Elvis Costello The Attractions
2,7 Years Too Late,61eaa7a2f02e1050431fb770,Tiffany Page
3,Accidents Will Happen,61eaa7a2f02e1050431fb771,Elvis Costello The Attractions
4,Alison,61eaa7a2f02e1050431fb772,Elvis Costello


In [31]:
users = users_coll.find()
users_csv = pd.DataFrame(data=users)
users_csv = users_csv.drop(['dateOfBirth', 'dateOfCreation', 'isAdmin', 'password', 'createdPlaylists'], axis=1)
users_csv.head()

,_id,username
0,61cc6cbb3bd8fb88c6628b8b,DeeCee
1,61cc6cbb3bd8fb88c6628b8c,CySorcerorD
2,61cc6cbb3bd8fb88c6628b8d,guesg
3,61cc6cbb3bd8fb88c6628b8e,SpaceGhostBenoit
4,61cc6cbb3bd8fb88c6628b8f,Meandmylife


In [32]:
users_csv['n_playlist_followed'] = [ random.randint(0,50)  for k in users_csv._id] 
users_csv['n_user_followed'] = [ random.randint(0,20)  for k in users_csv._id]
users_csv.head()

,_id,username,n_playlist_followed,n_user_followed
0,61cc6cbb3bd8fb88c6628b8b,DeeCee,21,0
1,61cc6cbb3bd8fb88c6628b8c,CySorcerorD,23,2
2,61cc6cbb3bd8fb88c6628b8d,guesg,36,17
3,61cc6cbb3bd8fb88c6628b8e,SpaceGhostBenoit,17,15
4,61cc6cbb3bd8fb88c6628b8f,Meandmylife,48,17


In [33]:
from neo4j import GraphDatabase

In [34]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [35]:
conn = Neo4jConnection(uri="neo4j://localhost:7687/browser/",user = "neo4j", pwd = "root") # Neo4j connection

In [36]:
# INSERT PLAYLIST NODES

for index, row in playlist_csv.iterrows():
    query_string = f'''
        CREATE (:Playlist {{name: "{row.playlist}", id: "{row.playlist_id}" }})
    '''
    conn.query(query_string,db='neo4j')

In [37]:
# INSERT USER NODES

for index, row in users_csv.iterrows():
    query_string = f'''
        CREATE (:User {{name: "{row.username}", id: "{row._id}" }})
    '''
    conn.query(query_string,db='neo4j')

In [38]:
# CREATE RELATIONSHIP
# User ->: LIKES -> Playlist

for index, row in users_csv.iterrows():
    for index2, row2 in playlist_csv.sample(row.n_playlist_followed).iterrows():
        query_string = f'''
        MATCH (u:User) WHERE u.id = "{row._id}"
        MATCH (p:Playlist) WHERE p.id = "{row2.playlist_id}"
        CREATE (u)-[:LIKES]->(p)
        '''

        conn.query(query_string,db='neo4j')

StopIteration: 

In [ ]:
# INSERT SONG NODES 

for index, row in songs_csv.iterrows():
    query_string = ''
    query_string = f'''
        CREATE (:Song {{
        id: "{row.song_id}",
        track: "{row.track}",
        artist: "{row.artist}"
        }})
    '''  
    conn.query(query_string,db='neo4j')

In [ ]:
# CREATE RELATIONSHIP
# User ->: FOLLOWS -> User

for index, row in users_csv.iterrows():
    for index2, row2 in users_csv.sample(row.n_user_followed).iterrows():
        query_string = f'''
        MATCH (u:User) WHERE u.id = "{row._id}"
        MATCH (u_2:User) WHERE u_2.id = "{row2._id}"
        CREATE (u)-[:FOLLOWS]->(u_2)
        '''
        #print(query_string)
        conn.query(query_string,db='neo4j')

In [ ]:
# Delete, if exists, self-reletionships from neo4j graph
query_string = f'''
        MATCH (a:User)-[rel:FOLLOWS]->(a2)
        WHERE a.id = a2.id
        DELETE rel;
        '''
conn.query(query_string,db='neo4j')

[]